In [1]:
import pandas as pd
import re
import numpy as np
import requests
import glob
import os
from tqdm import tqdm
from janitor import clean_names
from dicts import dict_columns_rrhh_download, dict_columns_select_download, path
from openpyxl import load_workbook, Workbook
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [17]:
def download_excel(files, path_download):
    """
        Extrae código, razon_social, rut y declaración de retención de impuestos y pagos provisionales codigos: 538, 142 y 020

        Args:
            files (str): path al archivo a extraer
    """
    
    try: 

        sheet_patron = r"BD_Postulación|BD_postulacion|BD postulacion|BD postulación"

        nombre_hoja = next(sheet for sheet in pd.ExcelFile(files).sheet_names if re.search(sheet_patron, sheet, flags=re.IGNORECASE))

        df = pd.read_excel(files,
                    sheet_name=nombre_hoja).clean_names()
        try:
            try:

                patron = "company_name|application_name"  # Patrón para identgantt_df_crea_r_r_r_creaicar la fila

                fila_nombres = df[df.apply(lambda row: row.str.contains(patron, case=False, na=False)).any(axis=1)].index[0]

            except:
                patron = "application_name"  # Patrón para identificar la fila

                fila_nombres = df[df.eq(patron).any(axis=1)].index[0]
            
            df = pd.read_excel(files,
                    sheet_name=nombre_hoja,
                    header=fila_nombres+1).clean_names()

            inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
                
            df = df.rename(columns=inverted_dict_column)

            df = df.filter(items=dict_columns_select_download)

        except:
            inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
            
            df = df.rename(columns=inverted_dict_column)

            df = df.filter(items=dict_columns_select_download)
    
    except:

        # sheet_patron = r"BD_Postulación|BD_postulacion|BD postulacion|BD postulación"

        # nombre_hoja = next(sheet for sheet in pd.ExcelFile(files).sheet_names if re.search(sheet_patron, sheet, flags=re.IGNORECASE))

        # df = pd.read_excel(files, sheet_name=sheet_patron).clean_names()

        df = pd.read_excel(files).clean_names()

        inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
            
        df = df.rename(columns=inverted_dict_column)

        #df = df[(df.application_sent == 'si') & (~df.plan_trabajo_presupuesto_rrhh.isna())]

        try:
            df = df.filter(items=dict_columns_select_download)

        except:
            df = df.loc[:, dict_columns_select_download]

    for index, row in df.iterrows():
        codigo = str(row['codigo'])
        carpeta_destino = f'{path_download}/{codigo}'

        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)

        hipervinculo = row['plan_trabajo_presupuesto_rrhh']
    
        nombre_archivo = codigo + '.xlsx'
        
        ruta_archivo = os.path.join(carpeta_destino, nombre_archivo)
        
        if os.path.exists(ruta_archivo):
            continue

        try:
            response = requests.get(hipervinculo)
            with open(ruta_archivo, 'wb') as file:
                file.write(response.content)
        except Exception as e:
            print(f"Error al descargar el archivo desde {hipervinculo}: {str(e)}")
            continue

In [18]:
path = r'data\1.2023\SN'

In [19]:
def rrhh_scrap(path, año=2023):
    """
        Recorre carpetas por año para extraer RRHH reportados por beneficiarios en base al código del proyecto

        Args:
            path (str): path a los archivos que contienen las BD de postulación con los archivos de RRHH postulados.
            año (int): año de la convocatoria
    """
    
    for concurso in tqdm(glob.iglob(f'{path}/*')):
        print(concurso)
        download_excel(concurso, path_download = 'output/1.2023/SN')

In [22]:
rrhh_scrap(path)

0it [00:00, ?it/s]

data\1.2023\SN\1.2023 SN BD Postulación.xlsx


1it [00:08,  8.72s/it]


# Cambios nombres de hoja de presupuesto

In [ ]:
path = r'data/2.2023/CH2/2.2023 Capital Humano Gestión de Evaluación.xlsx'

In [ ]:
df = pd.read_excel(path).clean_names()

In [ ]:
df.columns.to_list()

## **Proceso de Inserción de datos para la "CONFIGURACIÓN DE PROYECTOS" con UNA ETAPA.**

### **1. Preparación del perfil de usuario que ejecutará el código y las rutas de lectura de los archivos.**

In [2]:
# Se obtiene la ruta principal, que es llamada del directorio. Acá se pueden crear n rutas para distintos perfiles.
path['esteban']

'C:/Users/esteban.berrios/OneDrive - corfo.cl/'

In [3]:
# Tabla con presupuestos. El códigop permite generar nueva ruta llamada input_path que apunta a un directorio específico
# dentro de la estructura de directorios en path['esteban']. Esto facilita la manipulación y navegación en el sistema de archivos dentro de ese directorio en particular.
ruta = os.path.abspath(path['esteban'])
input_path = os.path.join(ruta, "extract_pdt/output/2.2023/IATS_PRUEBA")


In [4]:
input_path

'C:\\Users\\esteban.berrios\\OneDrive - corfo.cl\\extract_pdt/output/2.2023/IATS_PRUEBA'

In [5]:
#  Se define la variable folder como un vacío, y  con la siguiente línea de toma el primer archivo encontrado en una carpeta específica (dada por input_path y folder), 
# elimina la extensión ".xlsx" del nombre de ese archivo y devuelve el resultado, que probablemente sea utilizado como un identificador o código para ese archivo en particular.
folder = ""
os.listdir(os.path.join(input_path,folder))[0].replace(".xlsx","")

'2023-248257'

In [6]:
ingresa_fecha = input("Ingresa fecha de acuerdo a formato: DD-MM-YYYY: ")
print(ingresa_fecha)

01-08-2023


### **2. CARTA GANTT.**

#### **2.1 Ejecución de las actividades asociadas.**
Esta celda genera la creación de un archivo excel para cada proyecto sobre las actividades asociadas.

In [ ]:

# %%capture
# %%time
# ACTIVIDAD ETAPA CREA CON INSERCIÓN.
# PRUEBAS OK PARA SETEAR EL VALOR GUARDADO EN EL MÁXIMO  CONSIDERADO EN LA ETAPA CREA.
error_df_actividad = pd.DataFrame(columns=['Codigo', 'Error'])
max_mes = {}
fecha_valida_dict = {}

fecha_inicial = pd.to_datetime(ingresa_fecha, format='%d-%m-%Y')
# Bucle para procesar carpetas y archivos Excel
for folder in os.listdir(input_path):
    print(f"-------------------------------------------------------------------------------------------")
    print(f"entrando a {folder}")
    folder_path = os.path.join(input_path, folder)
    
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]
        print(f"leyendo {excel_files}")
        try:
            output_path = folder_path
            for excel_file in excel_files:
                file = os.path.join(folder_path, excel_file)
                codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

                excel_file_obj = pd.ExcelFile(file)
                sheet_names = excel_file_obj.sheet_names
                matching_sheets = [sheet_name for sheet_name in sheet_names if "PLAN DE TRABAJO" in sheet_name]
                
                if matching_sheets:
                    sheet_name = matching_sheets[0]
                    print(f"encontrando la hoja {sheet_name}")
                else:
                    sheet_name = None
                    print("Hoja no encontrada")

                if sheet_name is not None:
                    worksheet = pd.read_excel(file, sheet_name=sheet_name)
                    primera_fila = worksheet.index[worksheet.iloc[:, 2].astype(str).str.contains("Nombre")].tolist()
                    worksheet.columns = worksheet.iloc[0]
                    worksheet = worksheet.reset_index(drop=True)
                    primer_indice = primera_fila[0] if primera_fila else None
                    print(f"trayendo worksheet ubicado desde {primer_indice}")
                        #Delimita el rango de filas y columnas final con la cual se va a trabajar de la Etapa Crea.
                    if primer_indice is not None:
                        ws_crea = worksheet.iloc[primer_indice:,2:12].reset_index(drop=True)
                        ws_crea.columns = ws_crea.iloc[0]
                        ws_crea = ws_crea[1:].reset_index(drop=True)
                        print("trayendo worksheet delimitado")
                        file_nan = pd.isna(ws_crea.iloc[:, 0]).idxmax()
                        worksheet_crea = ws_crea.iloc[:file_nan].fillna(0)


                        etapa= np.array(['ETAPA','ETAPA 1','ETAPA 1','','','','','',''])

                        # Procedimiento para calcular las columnas "Fecha de Inicio" y "Fecha de Término"
                        worksheet_crea = worksheet_crea.dropna(subset=['Mes de Inicio', 'Mes de Término'])
                        worksheet_crea['Mes de Inicio'] = pd.to_numeric(worksheet_crea['Mes de Inicio'],
                                                                            errors="coerce").astype(int)
                        worksheet_crea['Mes de Término'] = pd.to_numeric(worksheet_crea['Mes de Término'],
                                                                            errors="coerce").astype(int)

                        max_valor_mes = worksheet_crea['Mes de Término'].max()
                        max_mes[codigo] = max_valor_mes

                        def calcular_fecha_inicial(row):
                            meses_a_sumar = max(0, row['Mes de Inicio'] - 1)
                            fecha_resultante = fecha_inicial + pd.DateOffset(months=meses_a_sumar)
                            return fecha_resultante.strftime('%d-%m-%Y')

                        def calcular_fecha_final(row):
                            fecha_final = fecha_inicial + pd.DateOffset(months=row['Mes de Término'])
                            return fecha_final.strftime('%d-%m-%Y')

                        gantt_df_actividad = pd.DataFrame({
                        # Asegúrate de que e_valida tenga un valor adecuado
                        'Tipo': 'ACTIVIDAD',
                        'Nombre': worksheet_crea.iloc[:,0].tolist(),
                        'Descripción / Observaciones': worksheet_crea.iloc[:,1].tolist(),
                        'Fecha de inicio': worksheet_crea.apply(calcular_fecha_inicial, axis=1),
                        'Fecha de término': worksheet_crea.apply(calcular_fecha_final, axis=1),
                        'Es crítica': '',
                        'Métricas': '',
                        'Etapa asociada': 'ETAPA 1',
                        'Actividad asociada': ''
                    })
                        nuevo_df = pd.DataFrame([etapa], columns=gantt_df_actividad.columns)
                        # Concatena el DataFrame temporal con el DataFrame original
                        gantt_df_actividad = pd.concat([nuevo_df, gantt_df_actividad], ignore_index=True)

                        # fecha_inicial_valida = fecha_inicial + relativedelta(months=max_valor_mes)
                        # fecha_inicial_valida_str = fecha_inicial_valida.strftime('%d-%m-%Y')
                        # fecha_valida_dict[codigo] = fecha_inicial_valida_str
                            
            output_filename = os.path.join(output_path, f"GANTT_{codigo}.xlsx")
            gantt_df_actividad.to_excel(output_filename, index=False, sheet_name="GANTT")

        except Exception as e:
            print(f'Error al abrir el archivo código: {codigo}: {str(e)}')
            max_mes[codigo] = 0
            gantt_df_actividad = pd.DataFrame({'Tipo':['Tipo'],'Nombre':['Nombre'],'Descripción / Observaciones':['Descripción / Observaciones'],
                                'Fecha de inicio':['Fecha de inicio'],'Fecha de término':['Fecha de término'],'Es crítica':['Es crítica'],
                                'Métricas':['Métricas'],'Etapa asociada':['Etapa asociada'],'Actividad asociada':['Actividad asociada']})
            etapa = np.array(['ETAPA','ETAPA CREA','ETAPA CREA','','','','','',''])
            nuevo_df = pd.DataFrame([etapa], columns=gantt_df_actividad.columns)
            # 2. Concatena el DataFrame temporal con el DataFrame original
            gantt_df_actividad = pd.concat([nuevo_df, gantt_df_actividad], ignore_index=True)
            gantt_df_actividad = gantt_df_actividad.drop(1, axis=0)
            gantt_df_actividad.reset_index(drop=True, inplace=True)

            exception_df = pd.DataFrame({'Codigo': [codigo], 'etapa':["Etapa Actividad"], 'Error': [str(e)]})
            error_df_actividad = pd.concat([error_df_actividad, exception_df], ignore_index=True)

            output_filename = os.path.join(output_path, f"GANTT_{codigo}.xlsx")
            gantt_df_actividad.to_excel(output_filename, index=False, sheet_name="GANTT")




#### **2.2 Ejecución de los resultados asociados**
Esta celda genera la inserción de los resultados para cada proyecto.

In [ ]:
# %%capture
# %%time
# OK RESULTADO ETAPA CREA CON INSERCIÓN
# PRUEBA OK PARA VERIFICAR CORRECTAMENTE EL VALOR MÁXIMO DE LOS PERÍODOS DE LA ETAPA CREA.
error_df_resultado = pd.DataFrame(columns=['Codigo', 'Error'])
patron = r'^GANTT_'

# Definir la variable fecha_inicial según tus necesidades
fecha_inicial = pd.to_datetime(ingresa_fecha, format='%d-%m-%Y')


# Inicializar la variable ruta_gantt como None
ruta_gantt = None

# Bucle para procesar carpetas y archivos Excel
for folder in os.listdir(input_path):
    print(f"-------------------------------------------------------------------------------------------")
    print(f"entrando a {folder}")
    folder_path = os.path.join(input_path, folder)
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]
        print(f"leyendo {excel_files}")
        try:
            output_path = folder_path

            for excel_file in excel_files:
                file = os.path.join(folder_path, excel_file)
                codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

                excel_file_obj = pd.ExcelFile(file)
                sheet_names = excel_file_obj.sheet_names
                matching_sheets = [sheet_name for sheet_name in sheet_names if "RESULTADO" in sheet_name]
                if matching_sheets:
                    sheet_name = matching_sheets[0]
                    print(f"encontrando la hoja {sheet_name}")
                else:
                    sheet_name = None
                    print("Hoja no encontrada")
                if sheet_name is not None:
                    print("Entra a la hoja RESULTADO")
                    worksheet_r = pd.read_excel(file, sheet_name=sheet_name)
                    worksheet_r = worksheet_r.iloc[5:, 1:7]
                    worksheet_r.columns = worksheet_r.iloc[0]
                    worksheet_r = worksheet_r[1:].reset_index(drop=True)

                    worksheet_r = worksheet_r.dropna(subset=['Mes de Obtención del resultado'])
                    worksheet_r['Mes de Obtención del resultado'] = pd.to_numeric(
                        worksheet_r['Mes de Obtención del resultado'],
                        errors="coerce").astype(int)
                    
                    # Obtener max_valor_crea del diccionario max_termino_proyecto usando el código del proyecto
                    max_valor_mes = max_mes.get(codigo, 0)

                    filtered_resultado = worksheet_r[worksheet_r['Mes de Obtención del resultado'] <= max_valor_mes]

                    def calcular_fecha_resultado(row):
                        fecha_final = fecha_inicial + pd.DateOffset(months=row['Mes de Obtención del resultado'])
                        return fecha_final.strftime('%d-%m-%Y')

                    gantt_df_resultado = pd.DataFrame({
                        # Asegúrate de que e_valida tenga un valor adecuado
                        'Tipo': 'RESULTADO',
                        'Nombre': filtered_resultado.iloc[:, 1].tolist(),
                        'Descripción / Observaciones': filtered_resultado.iloc[:, 2].tolist(),
                        'Fecha de inicio': '',
                        'Fecha de término': filtered_resultado.apply(calcular_fecha_resultado, axis=1),
                        'Es crítica': '',
                        'Métricas': filtered_resultado.iloc[:, 4].tolist(),
                        'Etapa asociada': "ETAPA 1",
                        'Actividad asociada': ''
                    })
                    gantt_df_resultado['Descripción / Observaciones'] = gantt_df_resultado['Descripción / Observaciones'].fillna(" ")

                # Verificar si el archivo cumple con el criterio
                if re.match(patron, excel_file):
                    # Si el archivo cumple con el criterio, asignar su ruta a ruta_gantt
                    ruta_gantt = os.path.join(folder_path, excel_file)
                    print(f"Ruta del archivo Excel que cumple con el criterio: {ruta_gantt}")
                    # Verificar nuevamente si el archivo Excel existe
                    if os.path.exists(ruta_gantt):
                        # Leer el archivo asociado a la ruta guardada de acuerdo al parámetro.
                        gantt_df = pd.read_excel(ruta_gantt)
                        # Encontrar la última fila no vacía en el DataFrame existente
                        last_row_index = gantt_df.index[-1] if not gantt_df.empty else 0
                        # Insertar el DataFrame gantt_df_crea_r debajo de la última fila
                        updated_df = pd.concat([gantt_df, gantt_df_resultado], ignore_index=True)
                        
                        # Guardar el DataFrame actualizado en el archivo Excel existente
                        updated_df.to_excel(ruta_gantt, index=False, sheet_name="GANTT")

        except Exception as e:
            print(f'Error al abrir el archivo código: {codigo}: {str(e)}')
            exception_df = pd.DataFrame({'Codigo': [codigo], 'etapa':["Etapa Resultado"], 'Error': [str(e)]})
            error_df_resultado = pd.concat([error_df_resultado, exception_df], ignore_index=True)

# Ahora puedes usar la variable ruta_gantt según tus necesidades
if ruta_gantt is not None:
    # Realiza las operaciones que necesites con ruta_gantt
    pass

#### **2.3 Ejecución de los errores generados en el proceso de lectura de la hoja de Plan de Trabajo para generar las cartas gantt.**
Esta celda genera la creación de un archivo excel que contiene los proyectos que tuvieron error de lectura. 

In [36]:
error_gantt_df = pd.concat([error_df_actividad, error_df_resultado], ignore_index=True)
output = os.path.join( input_path, "ERRORES_GANTT.xlsx")
error_gantt_df.to_excel(output, index = False, sheet_name="ERROR_GANTT")

### **3. GASTOS DE OPERACIÓN.**

#### **3.1 OPERACIÓN.**
Esta celda genera la creación de un archivo excel con el nombre "P_GASTOS_"código de proyecto".xlsx que contendrá el desglose de los gastos de operación para cada uno de los ítemes definidos.

In [ ]:
# OPERACIÓN ETAPA CREA OK REVISADO.

# Definir la variable fecha_inicial según tus necesidades
error_operacion = pd.DataFrame(columns=['Codigo','Error'])

fecha_inicial = pd.to_datetime(ingresa_fecha, format='%d-%m-%Y')

# Bucle para procesar carpetas y archivos Excel
for folder in os.listdir(input_path):
    print(f"-------------------------------------------------------------------------------------------")
    print(f"entrando a {folder}")
    folder_path = os.path.join(input_path, folder)
    
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]
        print(f"leyendo {excel_files}")
        try:
            output_path = folder_path
            operacion = pd.DataFrame()
            for excel_file in excel_files:
                file = os.path.join(folder_path, excel_file)
                codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

                excel_file_obj = pd.ExcelFile(file)
                sheet_names = excel_file_obj.sheet_names
                matching_sheets = [sheet_name for sheet_name in sheet_names if "OPERACION" in sheet_name]


                if matching_sheets:
                    sheet_name = matching_sheets[0]
                    print(f"encontrando la hoja {sheet_name}")
                else:
                    sheet_name = None
                    print("Hoja no encontrada")

                if sheet_name is not None:
                    worksheet = pd.read_excel(file, sheet_name=sheet_name)
                    primera_fila = worksheet.index[worksheet.iloc[:, 1].astype(str).str.contains("Ítem")].tolist()
                    worksheet.columns = worksheet.iloc[0]
                    worksheet = worksheet.reset_index(drop=True)
                    primer_indice = primera_fila[0] if primera_fila else None
                    print(f"trayendo worksheet ubicado desde {primer_indice}")



                        #Delimita el rango de filas y columnas final con la cual se va a trabajar de la Etapa Crea.
                    if primer_indice is not None:
                        ws = worksheet.iloc[primer_indice:,1:12].reset_index(drop=True)
                        ws.columns = ws.iloc[0]
                        ws = ws[1:].reset_index(drop=True)
                        print("trayendo worksheet delimitado")
                        file_nan = pd.isna(ws.iloc[:, 0]).idxmax()
                        worksheet_operacion = ws.iloc[:file_nan].fillna(0)

                        # Define el valor máximo del mes asociado a la etapa crea  a partir de la utilizacióin del diccionario.
                        max_valor_mes = max_mes.get(codigo, 0)



                        column = {
                            "Aporte Innova Chile\n(Subsidio) ($)": "CORFO",
                            "Aporte Beneficiaria (Pecuniario) ($)": "BENEFICIARIO",
                            "Aporte Beneficiaria (Valorado) ($)": "BENEFICIARIO",
                            "Aporte Asociados (Pecuniario) ($)": "ASOCIADO",
                            "Aporte Asociados (Valorado) ($)": "ASOCIADO"
                        }
            #             #--------
                        for index, row in worksheet_operacion.iterrows():
                            item = str(row.iloc[0])
                            descripcion_detalle = str(row.iloc[1])
                            valores = row.iloc[6:11]
                            
                            for columna, valor in zip(valores.index, valores):  
                                nombre_columna = column.get(columna)
                                #valores_divididos = [int(val / max_valor_crea) if val > 0 else 0 for val in valores]
                                valores_divididos = [int(valor / max_valor_mes) if valor > 0 else 0][0]


            #             #--------
                            # Crea el DataFrame utilizando las variables actualizadas
                                df_auxiliar = pd.DataFrame({
                                    'Cuenta': "Gastos de Operación",
                                    'Ítem': [item],
                                    'Descripción y detalle': [descripcion_detalle] ,
                                    'Fuente de financiamiento': [nombre_columna],
                                    "Total": valor,
                                    'Valor': valores_divididos,
                                })

                                

                                df_auxiliar = df_auxiliar[df_auxiliar['Valor'] > 0]

                                # Agrega el DataFrame auxiliar al DataFrame resultante
                                operacion = pd.concat([operacion, df_auxiliar], ignore_index=True)


                                operacion_df = pd.DataFrame()

                        # Itera a través de cada fila del DataFrame original

                        for index, row in operacion.iterrows():
                            repeticiones = int(max_valor_mes)
                            
                            # Replica la fila según el valor en 'max_valor_crea'
                            replicated_rows = pd.concat([row] * repeticiones, axis=1).T
                            suma_valor = replicated_rows['Valor'].sum()
                            # Obtén el valor en la primera fila de la columna "Total"
                            pvalor = replicated_rows['Total'].mean()
                            # Calcula la diferencia
                            diferencia = pvalor - suma_valor

                            if diferencia > 0:
                                # Sumar el valor de 'diferencia' con la primera fila correspondiente a 'replicated_rows'
                                primera_fila_replicated = replicated_rows.iloc[0]
                                primera_fila_replicated['Valor'] += diferencia

                                # Actualizar 'replicated_rows' con la fila modificada
                                replicated_rows.iloc[0] = primera_fila_replicated



                            fechas_incr = [fecha_inicial + relativedelta(months=i) for i in range(repeticiones)]
                            # fechas_con_formato = [fecha.strftime('%d-%m-%Y') for fecha in fechas_incr]
                            operacion_df = pd.concat([operacion_df, replicated_rows], ignore_index=True)
                            # Agrega la columna 'fecha' con las fechas con formato a replicated_rows
                            replicated_rows['Fecha'] = fechas_incr#fechas_con_formato
                            # Agrega replicated_rows al DataFrame operacion_df
                            operacion_df = pd.concat([operacion_df, replicated_rows], ignore_index=True)
                            operacion_df = operacion_df.dropna(subset=['Fecha'])
                            # Genera una columna 'año' que contiene el año de la fecha
                            operacion_df['Año'] = operacion_df['Fecha'].dt.year

                            # Genera una columna 'mes' que contiene el mes de la fecha
                            operacion_df['Mes'] = operacion_df['Fecha'].dt.month
                            operacion_df['Mes'] = operacion_df['Mes'].astype(int)
                            meses_n = {1: 'Ene',2: 'Feb',3: 'Mar',4: 'Abr',5: 'May',6: 'Jun',
                                7: 'Jul',8: 'Ago',9: 'Sep',10: 'Oct',11: 'Nov',12: 'Dic'}

                            # Aplica la función de mapeo
                            operacion_df['Mes'] = operacion_df['Mes'].map(meses_n)

                            

            operacion_df = operacion_df.drop(columns=["Total", "Fecha"])
            operacion_df = operacion_df[[col for col in operacion_df if col != 'Valor'] + ['Valor']]
                               
            output_xlsx = os.path.join(output_path, f"P_GASTOS_{codigo}.xlsx")
            operacion_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")                            
        
        except Exception as e:
            print(f'Error al abrir el archivo código: {codigo}: {str(e)}')
            max_mes[codigo] = 0
            operacion_df = pd.DataFrame({'Cuenta':['Cuenta'],'Ítem':['Ítem'],'Descripción y detalle':['Descripción y detalle'],
                                              'Fuente de financiamiento':['Fuente de financiamiento'],'Año':['Año'],'Mes':['Mes'],'Valor':['Valor']})
            empty_array = np.array(['','','','','','',''])
            new_df = pd.DataFrame([empty_array], columns=operacion_df.columns)
            operacion_df = pd.concat([new_df, operacion_df], ignore_index=True)
            operacion_df = operacion_df.drop(1,axis=0)
            operacion_df.reset_index(drop=True, inplace=True)
            
            excepcion_gastos_df = pd.DataFrame({'Codigo': [codigo], 'Etapa':["Operación"], 'Error': [str(e)]})
            error_operacion = pd.concat([error_operacion, excepcion_gastos_df], ignore_index=True)

            output_xlsx = os.path.join(output_path, f"P_GASTOS_{codigo}.xlsx")
            operacion_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")


#### **3.2 Ejecución de los errores generados en el proceso de lectura de la hoja de operación.**
Esta celda genera la creación de un archivo excel que contiene los proyectos que tuvieron error de lectura. 

In [65]:
output = os.path.join( input_path, "ERRORES_OPERACION.xlsx")
error_operacion.to_excel(output, index = False, sheet_name="ERROR_OPERACION")

### **4. RRHH.**

#### **4.1 RRHH.**
Esta celda genera la creación de un archivo excel con el nombre "RRHH_"código de proyecto".xlsx que contendrá el desglose de los gastos por concepto de RRHH para cada uno de los ítemes definidos. 

In [ ]:
# RRHH ETAPA CREA OK REVISADO.

# Definir la variable fecha_inicial según tus necesidades
error_rrhh = pd.DataFrame(columns=['Codigo','Error'])

fecha_inicial = pd.to_datetime(ingresa_fecha, format='%d-%m-%Y')

# Bucle para procesar carpetas y archivos Excel
for folder in os.listdir(input_path):
    print(f"-------------------------------------------------------------------------------------------")
    print(f"entrando a {folder}")
    folder_path = os.path.join(input_path, folder)
    
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]
        print(f"leyendo {excel_files}")
        try:
            output_path = folder_path
            rrhh = pd.DataFrame()
            for excel_file in excel_files:
                file = os.path.join(folder_path, excel_file)
                codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

                excel_file_obj = pd.ExcelFile(file)
                sheet_names = excel_file_obj.sheet_names
                matching_sheets = [sheet_name for sheet_name in sheet_names if "RRHH" in sheet_name]


                if matching_sheets:
                    sheet_name = matching_sheets[0]
                    print(f"encontrando la hoja {sheet_name}")
                else:
                    sheet_name = None
                    print("Hoja no encontrada")

                if sheet_name is not None:
                    worksheet = pd.read_excel(file, sheet_name=sheet_name)
                    primera_fila = worksheet.index[worksheet.iloc[:, 1].astype(str).str.contains("Nombre")].tolist()
                    worksheet.columns = worksheet.iloc[0]
                    worksheet = worksheet.reset_index(drop=True)
                    primer_indice = primera_fila[0] if primera_fila else None
                    print(f"trayendo worksheet ubicado desde {primer_indice}")

                    
                    #Delimita el rango de filas y columnas final con la cual se va a trabajar de la Etapa Crea.
                    if primer_indice is not None:
                        ws = worksheet.iloc[primer_indice:,1:19].reset_index(drop=True)
                        ws.columns = ws.iloc[0]
                        ws = ws[1:].reset_index(drop=True)
                        print("trayendo worksheet delimitado")
                        file_nan = pd.isna(ws.iloc[:, 0]).idxmax()
                        worksheet_rrhh = ws.iloc[:file_nan].fillna(0)

    #                     # Define el valor máximo del mes asociado a la etapa crea  a partir de la utilizacióin del diccionario.
    #                     max_valor_crea = max_termino_crea.get(codigo, 0)



                        column = {
                            "Aporte Innova Chile\n(Subsidio) ($)": "CORFO",
                            "Aporte Beneficiaria (Pecuniario) ($)": "BENEFICIARIO",
                            "Aporte Beneficiaria (Valorado) ($)": "BENEFICIARIO",
                            "Aporte Asociados (Pecuniario) ($)": "ASOCIADO",
                            "Aporte Asociados (Valorado) ($)": "ASOCIADO"
                        }
    #                     #--------
                        for index, row in worksheet_rrhh.iterrows():
                            funcionario = str(row.iloc[0])
                            rut = str(row.iloc[1])
                            cargo = str(row.iloc[5])
                            descripcion = str(row.iloc[7])
                            meses = (row.iloc[10])
                            valor_hora = (row.iloc[12])
                            valores = row.iloc[13:18]
                            
                            for columna, valor in zip(valores.index, valores):  
                                nombre_columna = column.get(columna)
                                #valores_divididos = [int(val / max_valor_crea) if val > 0 else 0 for val in valores]
                                valores_divididos = [int(valor / meses) if valor > 0 else 0][0]


    #                     #--------
                            # Crea el DataFrame utilizando las variables actualizadas
                                df_auxiliar = pd.DataFrame({
                                    'Cuenta': "Gastos RRHH",
                                    'Ítem': [funcionario],
                                    'Rut': [rut],
                                    'Descripción': [descripcion],
                                    'Meses': [meses],
                                    'Valor Hora': [valor_hora],
                                    'Fuente de financiamiento': [nombre_columna],
                                    "Total": valor,
                                    'Valor': valores_divididos,
                                })

                                

                                df_auxiliar = df_auxiliar[df_auxiliar['Valor'] > 0]

                                # Agrega el DataFrame auxiliar al DataFrame resultante
                                rrhh = pd.concat([rrhh, df_auxiliar], ignore_index=True)


                                rrhh_df = pd.DataFrame()

    #                     # Itera a través de cada fila del DataFrame original

                        for index, row in rrhh.iterrows():
                            repeticiones = int(meses)
                            
                            # Replica la fila según el valor en 'max_valor_crea'
                            replicated_rows = pd.concat([row] * repeticiones, axis=1).T
                            suma_valor = replicated_rows['Valor'].sum()
                            # Obtén el valor en la primera fila de la columna "Total"
                            pvalor = replicated_rows['Total'].mean()
                            # Calcula la diferencia
                            diferencia = pvalor - suma_valor

                            if diferencia > 0:
                                # Sumar el valor de 'diferencia' con la primera fila correspondiente a 'replicated_rows'
                                primera_fila_replicated = replicated_rows.iloc[0]
                                primera_fila_replicated['Valor'] += diferencia

                                # Actualizar 'replicated_rows' con la fila modificada
                                replicated_rows.iloc[0] = primera_fila_replicated



                            fechas_incr = [fecha_inicial + relativedelta(months=i) for i in range(repeticiones)]
                            # fechas_con_formato = [fecha.strftime('%d-%m-%Y') for fecha in fechas_incr]
                            rrhh_df = pd.concat([rrhh_df, replicated_rows], ignore_index=True)
                            # Agrega la columna 'fecha' con las fechas con formato a replicated_rows
                            replicated_rows['Fecha'] = fechas_incr#fechas_con_formato
                            # Agrega replicated_rows al DataFrame operacion_crea_df
                            rrhh_df = pd.concat([rrhh_df, replicated_rows], ignore_index=True)
                            rrhh_df = rrhh_df.dropna(subset=['Fecha'])
                            # Genera una columna 'año' que contiene el año de la fecha
                            rrhh_df['Año'] = rrhh_df['Fecha'].dt.year

                            # Genera una columna 'mes' que contiene el mes de la fecha
                            rrhh_df['Mes'] = rrhh_df['Fecha'].dt.month
                            rrhh_df['Mes'] = rrhh_df['Mes'].astype(int)
                            meses_n = {1: 'Ene',2: 'Feb',3: 'Mar',4: 'Abr',5: 'May',6: 'Jun',
                                7: 'Jul',8: 'Ago',9: 'Sep',10: 'Oct',11: 'Nov',12: 'Dic'}

                            # Aplica la función de mapeo
                            rrhh_df['Mes'] = rrhh_df['Mes'].map(meses_n)

                                

            rrhh_df = rrhh_df.drop(columns=["Total", "Fecha"])
            rrhh_df = rrhh_df[[col for col in rrhh_df if col != 'Valor'] + ['Valor']]

                               
            # output_xlsx = os.path.join(output_path, f"RRHH_{codigo}.xlsx")
            # rrhh_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")                            
        
        except Exception as e:
            print(f'Error al abrir el archivo código: {codigo}: {str(e)}')
    
            rrhh_df = pd.DataFrame({'Cuenta':['Cuenta'],'Ítem':['Ítem'],'Rut':['Rut'],'Descripción':['Descripción'],
                                         'Meses':['Meses'],'Valor Hora':['Valor Hora'],'Fuente de financiamiento':['Fuente de financiamiento'],
                                         'Valor':['Valor'],'Año':['Año'],'Mes':['Mes'] })
            empty_array = np.array(['','','','','','','','','',''])
            new_df = pd.DataFrame([empty_array], columns=rrhh_df.columns)
            rrhh_df = pd.concat([new_df, rrhh_df], ignore_index=True)
            rrhh_df = rrhh_df.drop(1,axis=0)
            rrhh_df.reset_index(drop=True, inplace=True)
            
            excepcion_rrhh_df = pd.DataFrame({'Codigo': [codigo], 'Etapa':["RRHH"], 'Error': [str(e)]})
            error_rrhh = pd.concat([error_rrhh, excepcion_rrhh_df], ignore_index=True)

                
            # output_xlsx = os.path.join(output_path, f"RRHH_{codigo}.xlsx")
            # rrhh_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")


#### **4.2 Ejecución de los errores generados en el proceso de lectura de la hoja de RRHH.**
Esta celda genera la creación de un archivo excel que contiene los proyectos que tuvieron error de lectura. 

In [ ]:
output = os.path.join( input_path, "ERRORES_RRHH.xlsx")
error_rrhh.to_excel(output, index = False, sheet_name="ERROR_RRHH")

### **5. ADMINISTRACIÓN.**

#### **5.1 ADMINISTRACIÓN.**
Esta celda genera la creación de un archivo excel con el nombre "ADM_"código de proyecto".xlsx que contendrá el desglose de los gastos por concepto de ADMINISTRACIÓN para cada uno de los ítemes definidos.

In [ ]:
# ADMINISTRACIÓN ETAPA CREA OK REVISADO.

# Definir la variable fecha_inicial según tus necesidades
error_adm = pd.DataFrame(columns=['Codigo','Error'])

fecha_inicial = pd.to_datetime(ingresa_fecha, format='%d-%m-%Y')

# Bucle para procesar carpetas y archivos Excel
for folder in os.listdir(input_path):
    print(f"-------------------------------------------------------------------------------------------")
    print(f"entrando a {folder}")
    folder_path = os.path.join(input_path, folder)
    
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]
        print(f"leyendo {excel_files}")
        try:
            output_path = folder_path
            administracion = pd.DataFrame()
            for excel_file in excel_files:
                file = os.path.join(folder_path, excel_file)
                codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

                excel_file_obj = pd.ExcelFile(file)
                sheet_names = excel_file_obj.sheet_names
                matching_sheets = [sheet_name for sheet_name in sheet_names if "ADMINISTRACIÓN" in sheet_name]


                if matching_sheets:
                    sheet_name = matching_sheets[0]
                    print(f"encontrando la hoja {sheet_name}")
                else:
                    sheet_name = None
                    print("Hoja no encontrada")

                if sheet_name is not None:
                    worksheet = pd.read_excel(file, sheet_name=sheet_name)
                    primera_fila = worksheet.index[worksheet.iloc[:, 1].astype(str).str.contains("Ítem")].tolist()
                    worksheet.columns = worksheet.iloc[0]
                    worksheet = worksheet.reset_index(drop=True)
                    primer_indice = primera_fila[0] if primera_fila else None
                    print(f"trayendo worksheet ubicado desde {primer_indice}")                    
                    


                    #Delimita el rango de filas y columnas final con la cual se va a trabajar de la Etapa Crea.
                    if primer_indice is not None:
                        ws_crea = worksheet.iloc[primer_indice:, 1:9].reset_index(drop=True)
                        ws_crea.columns = ws_crea.iloc[0]
                        ws_crea = ws_crea[1:].reset_index(drop=True)
                        print("trayendo worksheet delimitado")
                        file_nan = pd.isna(ws_crea.iloc[:, 0]).idxmax()
                        worksheet_crea = ws_crea.iloc[:file_nan].fillna(0)

                        # Define el valor máximo del mes asociado a la etapa crea  a partir de la utilizacióin del diccionario.
                        max_valor_mes = max_mes.get(codigo, 0)



                        column = {
                            "Aporte Innova Chile\n(Subsidio) ($)": "CORFO",
                            "Aporte Beneficiaria (Pecuniario) ($)": "BENEFICIARIO",
                            "Aporte Beneficiaria (Valorado) ($)": "BENEFICIARIO",
                            "Aporte Asociados (Pecuniario) ($)": "ASOCIADO",
                            "Aporte Asociados (Valorado) ($)": "ASOCIADO"
                        }
                        #--------
                        for index, row in worksheet_crea.iterrows():
                            item = str(row.iloc[0])
                            descripcion_detalle = str(row.iloc[1])
                            valores = row.iloc[4:9]
                            
                            for columna, valor in zip(valores.index, valores):  
                                nombre_columna = column.get(columna)
                                #valores_divididos = [int(val / max_valor_crea) if val > 0 else 0 for val in valores]
                                valores_divididos = [int(valor / max_valor_mes) if valor > 0 else 0][0]


                        #--------
                            # Crea el DataFrame utilizando las variables actualizadas
                                df_auxiliar = pd.DataFrame({
                                    'Cuenta': "Gastos de Administración",
                                    'Ítem': [item],
                                    'Descripción y detalle': [descripcion_detalle] ,
                                    'Fuente de financiamiento': [nombre_columna],
                                    "Total": valor,
                                    'Valor': valores_divididos,
                                })

                                

                                df_auxiliar = df_auxiliar[df_auxiliar['Valor'] > 0]

                                # Agrega el DataFrame auxiliar al DataFrame resultante
                                administracion = pd.concat([administracion, df_auxiliar], ignore_index=True)


                                administracion_df = pd.DataFrame()

                        # Itera a través de cada fila del DataFrame original

                        for index, row in administracion.iterrows():
                            repeticiones = int(max_valor_mes)
                            
                            # Replica la fila según el valor en 'max_valor_crea'
                            replicated_rows = pd.concat([row] * repeticiones, axis=1).T
                            suma_valor = replicated_rows['Valor'].sum()
                            # Obtén el valor en la primera fila de la columna "Total"
                            pvalor = replicated_rows['Total'].mean()
                            # Calcula la diferencia
                            diferencia = pvalor - suma_valor

                            if diferencia > 0:
                                # Sumar el valor de 'diferencia' con la primera fila correspondiente a 'replicated_rows'
                                primera_fila_replicated = replicated_rows.iloc[0]
                                primera_fila_replicated['Valor'] += diferencia

                                # Actualizar 'replicated_rows' con la fila modificada
                                replicated_rows.iloc[0] = primera_fila_replicated



                            fechas_incr = [fecha_inicial + relativedelta(months=i) for i in range(repeticiones)]
                            # fechas_con_formato = [fecha.strftime('%d-%m-%Y') for fecha in fechas_incr]
                            administracion_df = pd.concat([administracion_df, replicated_rows], ignore_index=True)
                            # Agrega la columna 'fecha' con las fechas con formato a replicated_rows
                            replicated_rows['Fecha'] = fechas_incr#fechas_con_formato
                            # Agrega replicated_rows al DataFrame operacion_crea_df
                            administracion_df = pd.concat([administracion_df, replicated_rows], ignore_index=True)
                            administracion_df = administracion_df.dropna(subset=['Fecha'])
                            # Genera una columna 'año' que contiene el año de la fecha
                            administracion_df['Año'] = administracion_df['Fecha'].dt.year

                            # Genera una columna 'mes' que contiene el mes de la fecha
                            administracion_df['Mes'] = administracion_df['Fecha'].dt.month
                            administracion_df['Mes'] = administracion_df['Mes'].astype(int)
                            meses_n = {1: 'Ene',2: 'Feb',3: 'Mar',4: 'Abr',5: 'May',6: 'Jun',
                                7: 'Jul',8: 'Ago',9: 'Sep',10: 'Oct',11: 'Nov',12: 'Dic'}

                            # Aplica la función de mapeo
                            administracion_df['Mes'] = administracion_df['Mes'].map(meses_n)

                                

            administracion_df = administracion_df.drop(columns=["Total", "Fecha"])
            administracion_df = administracion_df[[col for col in administracion_df if col != 'Valor'] + ['Valor']]
                               
            output_xlsx = os.path.join(output_path, f"ADM_{codigo}.xlsx")
            administracion_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")                            
        
        except Exception as e:
            print(f'Error al abrir el archivo código: {codigo}: {str(e)}')
            # max_termino_crea[codigo] = 0
            administracion_df = pd.DataFrame({'Cuenta':['Cuenta'],'Ítem':['Ítem'],'Descripción y detalle':['Descripción y detalle'],
                                              'Fuente de financiamiento':['Fuente de financiamiento'],'Año':['Año'],'Mes':['Mes'],'Valor':['Valor']})
            empty_array = np.array(['','','','','','',''])
            new_df = pd.DataFrame([empty_array], columns=administracion_df.columns)
            administracion_df = pd.concat([new_df, administracion_df], ignore_index=True)
            administracion_df = administracion_df.drop(1,axis=0)
            administracion_df.reset_index(drop=True, inplace=True)
            
            excepcion_adm_df = pd.DataFrame({'Codigo': [codigo], 'Etapa':["Administración"], 'Error': [str(e)]})
            error_adm = pd.concat([error_adm, excepcion_adm_df], ignore_index=True)

            output_xlsx = os.path.join(output_path, f"ADM_{codigo}.xlsx")
            administracion_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")


#### **5.2 Ejecución de los errores generados en el proceso de lectura de la hoja de administracióbn.**
Esta celda genera la creación de un archivo excel que contiene los proyectos que tuvieron error de lectura. 

In [98]:
output = os.path.join( input_path, "ERRORES_ADM.xlsx")
error_adm.to_excel(output, index = False, sheet_name="ERROR_ADM")


### **6. INVERSIÓN.**

#### **6.1 INVERSIÓN ETAPA CREA.**
Esta celda genera la creación de un archivo excel con el nombre "INV_"código de proyecto".xlsx que contendrá el desglose de los gastos por concepto de INVERSIÓN para cada uno de los ítemes definidos. 

In [ ]:
# INVERSIÓN ETAPA CREA OK REVISADO.

# Definir la variable fecha_inicial según tus necesidades
error_inversion = pd.DataFrame(columns=['Codigo','Error'])

fecha_inicial = pd.to_datetime(ingresa_fecha, format='%d-%m-%Y')

# Bucle para procesar carpetas y archivos Excel
for folder in os.listdir(input_path):
    print(f"-------------------------------------------------------------------------------------------")
    print(f"entrando a {folder}")
    folder_path = os.path.join(input_path, folder)
    
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]
        print(f"leyendo {excel_files}")
        try:
            output_path = folder_path
            inversion = pd.DataFrame()
            # inversion_crea_df = pd.DataFrame()
            for excel_file in excel_files:
                file = os.path.join(folder_path, excel_file)
                codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

                excel_file_obj = pd.ExcelFile(file)
                sheet_names = excel_file_obj.sheet_names
                matching_sheets = [sheet_name for sheet_name in sheet_names if "INVERSIÓN" in sheet_name]


                if matching_sheets:
                    sheet_name = matching_sheets[0]
                    print(f"encontrando la hoja {sheet_name}")
                else:
                    sheet_name = None
                    print("Hoja no encontrada")

                if sheet_name is not None:
                    worksheet = pd.read_excel(file, sheet_name=sheet_name)
                    primera_fila = worksheet.index[worksheet.iloc[:, 1].astype(str).str.contains("Descripción")].tolist()
                    worksheet.columns = worksheet.iloc[0]
                    worksheet = worksheet.reset_index(drop=True)
                    primer_indice = primera_fila[0] if primera_fila else None
                    print(f"trayendo worksheet ubicado desde {primer_indice}")                    
    


                    if primer_indice is not None:
                        ws = worksheet.iloc[primer_indice:, 1:10].reset_index(drop=True)
                        ws.columns = ws.iloc[0]
                        ws = ws[1:].reset_index(drop=True)
                        print("trayendo worksheet delimitado")
                        file_nan = pd.isna(ws.iloc[:, 0]).idxmax()
                        worksheet_inversion = ws.iloc[:file_nan].fillna(0)

                        # Define el valor máximo del mes asociado a la etapa crea  a partir de la utilizacióin del diccionario.
                        max_valor_mes = max_mes.get(codigo, 0)



                        column = {
                            "Aporte Innova Chile\n(Subsidio) ($)": "CORFO",
                            "Aporte Beneficiaria (Pecuniario) ($)": "BENEFICIARIO",
                            "Aporte Beneficiaria (Valorado) ($)": "BENEFICIARIO",
                            "Aporte Asociados (Pecuniario) ($)": "ASOCIADO",
                            "Aporte Asociados (Valorado) ($)": "ASOCIADO"
                        }
                        #--------
                        for index, row in worksheet_inversion.iterrows():
                            item = str(row.iloc[0])
                            descripcion_detalle = str(row.iloc[1])
                            valores = row.iloc[4:10]
                            
                            for columna, valor in zip(valores.index, valores):  
                                nombre_columna = column.get(columna)
                                #valores_divididos = [int(val / max_valor_crea) if val > 0 else 0 for val in valores]
                                valores_divididos = [int(valor / max_valor_mes) if valor > 0 else 0][0]


                        #--------
                            # Crea el DataFrame utilizando las variables actualizadas
                                df_auxiliar = pd.DataFrame({
                                    'Cuenta': "Gastos de Inversión",
                                    'Ítem': [item],
                                    'Descripción y detalle': [descripcion_detalle] ,
                                    'Fuente de financiamiento': [nombre_columna],
                                    "Total": valor,
                                    'Valor': valores_divididos,
                                })

                                

                                df_auxiliar = df_auxiliar[df_auxiliar['Valor'] > 0]

                                # Agrega el DataFrame auxiliar al DataFrame resultante
                                inversion = pd.concat([inversion, df_auxiliar], ignore_index=True)


                                inversion_df = pd.DataFrame()

        #                 # Itera a través de cada fila del DataFrame original

                        for index, row in inversion.iterrows():
                            repeticiones = int(max_valor_mes)
                            
                            # Replica la fila según el valor en 'max_valor_crea'
                            replicated_rows = pd.concat([row] * repeticiones, axis=1).T
                            suma_valor = replicated_rows['Valor'].sum()
                            # Obtén el valor en la primera fila de la columna "Total"
                            pvalor = replicated_rows['Total'].mean()
                            # Calcula la diferencia
                            diferencia = pvalor - suma_valor

                            if diferencia > 0:
                                # Sumar el valor de 'diferencia' con la primera fila correspondiente a 'replicated_rows'
                                primera_fila_replicated = replicated_rows.iloc[0]
                                primera_fila_replicated['Valor'] += diferencia

                                # Actualizar 'replicated_rows' con la fila modificada
                                replicated_rows.iloc[0] = primera_fila_replicated



                            fechas_incr = [fecha_inicial + relativedelta(months=i) for i in range(repeticiones)]
                            # fechas_con_formato = [fecha.strftime('%d-%m-%Y') for fecha in fechas_incr]
                            inversion_df = pd.concat([inversion_df, replicated_rows], ignore_index=True)
                            # Agrega la columna 'fecha' con las fechas con formato a replicated_rows
                            replicated_rows['Fecha'] = fechas_incr#fechas_con_formato
                            # Agrega replicated_rows al DataFrame inversion_df
                            inversion_df = pd.concat([inversion_df, replicated_rows], ignore_index=True)
                            inversion_df = inversion_df.dropna(subset=['Fecha'])
                            # Genera una columna 'año' que contiene el año de la fecha
                            inversion_df['Año'] = inversion_df['Fecha'].dt.year

                            # Genera una columna 'mes' que contiene el mes de la fecha
                            inversion_df['Mes'] = inversion_df['Fecha'].dt.month
                            inversion_df['Mes'] = inversion_df['Mes'].astype(int)
                            meses_n = {1: 'Ene',2: 'Feb',3: 'Mar',4: 'Abr',5: 'May',6: 'Jun',
                                7: 'Jul',8: 'Ago',9: 'Sep',10: 'Oct',11: 'Nov',12: 'Dic'}

                            # Aplica la función de mapeo
                            inversion_df['Mes'] = inversion_df['Mes'].map(meses_n)

                            

            inversion_df = inversion_df.drop(columns=["Total", "Fecha"])
            inversion_df = inversion_df[[col for col in inversion_df if col != 'Valor'] + ['Valor']]
                               
            output_xlsx = os.path.join(output_path, f"INV_{codigo}.xlsx")
            inversion_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")                            
        
        except Exception as e:
            print(f'Error al abrir el archivo código: {codigo}: {str(e)}')
            inversion_df = pd.DataFrame({'Cuenta':['Cuenta'],'Ítem':['Ítem'],'Descripción y detalle':['Descripción y detalle'],
                                              'Fuente de financiamiento':['Fuente de financiamiento'],'Año':['Año'],'Mes':['Mes'],'Valor':['Valor']})
            empty_array = np.array(['','','','','','',''])
            new_df = pd.DataFrame([empty_array], columns=inversion_df.columns)
            inversion_df = pd.concat([new_df, inversion_df], ignore_index=True)
            inversion_df = inversion_df.drop(1,axis=0)
            inversion_df.reset_index(drop=True, inplace=True)
            
            excepcion_inv_df = pd.DataFrame({'Codigo': [codigo], 'Etapa':["Inversión"], 'Error': [str(e)]})
            error_inversion = pd.concat([error_inversion, excepcion_inv_df], ignore_index=True)

            output_xlsx = os.path.join(output_path, f"INV_{codigo}.xlsx")
            inversion_df.to_excel(output_xlsx, index=False, sheet_name="ETAPA 1")

#### **6.2 Ejecución de los errores generados en el proceso de lectura de la hoja de INVERSIÓN.**
Esta celda genera la creación de un archivo excel que contiene los proyectos que tuvieron error de lectura. 

In [111]:
output = os.path.join( input_path, "ERRORES_INV.xlsx")
error_inversion.to_excel(output, index = False, sheet_name="ERROR_INV")